<a href="https://colab.research.google.com/github/theouterlimitz/stockpricepredictor/blob/main/future_price_prediction_using_neural_network_and_place_it_into_a_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Download historical stock data
ticker = "TSLA"
start_date = "2020-01-01"
end_date = "2024-02-05"
data = yf.download(ticker, start=start_date, end=end_date)

# Preprocess data for LSTM
def prepare_data(data, window_size, train_pct=0.8):
    prices = data["Close"]
    num_train = int(len(prices) * train_pct)
    train, test = prices[:num_train], prices[num_train:]

    # Create sequences for LSTM
    def create_sequences(data, seq_length):
        sequences = []
        for i in range(len(data) - seq_length):
            sequences.append(data[i:i + seq_length])
        return np.array(sequences)

    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test.values.reshape(-1, 1))

    # Create sequences
    train_sequences = create_sequences(train_scaled, window_size)
    test_sequences = create_sequences(test_scaled, window_size)

    return train_sequences, test_sequences, scaler

window_size = 32
train_sequences, test_sequences, scaler = prepare_data(data, window_size)

# Build LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(window_size, 1)))
model.add(LSTM(100))
model.add(Dense(3))
model.add(Dense(1))
model.add(Dense(3))
model.add(Dense(1))
model.compile(loss="mae", optimizer="adam")

# Train model
model.fit(train_sequences, train_sequences[:, -1], epochs=100, batch_size=16)

# Make predictions on test data
predicted_prices = []
for seq in test_sequences:
    predicted_price = model.predict(seq.reshape(1, seq.shape[0], 1))[0][0]
    predicted_prices.append(scaler.inverse_transform([[predicted_price]])[0][0])

# Make predictions on future data
future_dates = pd.date_range(data.index[-1] + pd.DateOffset(1), periods=45)  # 30 days into the future
future_prices = []
last_window = test_sequences[-1]  # Use the last window from test data as input for future predictions
for i in range(len(future_dates)):
    predicted_price = model.predict(last_window.reshape(1, window_size, 1))[0][0]
    future_prices.append(scaler.inverse_transform([[predicted_price]])[0][0])
    last_window = np.append(last_window[1:], predicted_price)  # Update the window for the next prediction

# Visualize results
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["Close"], label="Actual")
plt.plot(data.index[-len(predicted_prices):], predicted_prices, label="Predicted (Test Data)")
plt.plot(future_dates, future_prices, label="Predicted (Future)")
plt.title("TLSA Stock Price Prediction")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

**Key Changes:**

- **Future data prediction:** Added code to generate predictions for 30 days into the future, using the last window from the test data as the initial input.
- **Visualization:** Included